### Load Packages

In [42]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper
import project_tests
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver import EdgeOptions
from parser_10KQ import get_word_list
from parser_10KQ import lemmatize_words
from itertools import islice
# options = EdgeOptions()
# options.add_argument("--headless")
# options.add_argument("--window-size=%s" % "3840, 2160")

In [28]:
with open('fillings_by_ticker 1', 'rb') as handle:
    fillings_by_ticker = pickle.load(handle)

In [4]:
with open('filling_lem', 'rb') as handle:
    filling_lem = pickle.load(handle)

In [28]:
sentiment_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
sentiment_df.columns = [column.lower() for column in sentiment_df.columns]

In [136]:
sentiment_df

,word,seq_num,word count,word proportion,average proportion,std dev,doc count,negative,positive,uncertainty,litigious,strong_modal,weak_modal,constraining,syllables,source
0,AARDVARK,1,354,0.00000002,0.00000001,0.00000382,99,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,0.00000000,0.00000000,0.00000001,1,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,0.00000000,0.00000000,0.00000005,7,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,0.00000000,0.00000000,0.00000016,28,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8570,0.00000038,0.00000038,0.00003529,1108,0,0,0,0,0,0,0,3,12of12inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86526,ZYGOTE,86529,50,0.00000000,0.00000000,0.00000019,35,0,0,0,0,0,0,0,2,12of12inf
86527,ZYGOTES,86530,1,0.00000000,0.00000000,0.00000002,1,0,0,0,0,0,0,0,2,12of12inf
86528,ZYGOTIC,86531,0,0.00000000,0.00000000,0.00000000,0,0,0,0,0,0,0,0,3,12of12inf
86529,ZYMURGIES,86532,0,0.00000000,0.00000000,0.00000000,0,0,0,0,0,0,0,0,3,12of12inf


In [50]:
sentiment_neg = sentiment_df[['word']][sentiment_df["negative"]>0]

In [62]:
sentiment_neg['word'] = lemmatize_words(sentiment_neg['word'].str.lower())
sentiment_neg = sentiment_neg.drop_duplicates('word')

In [64]:
word_list = sentiment_neg['word'].str.lower().values

In [165]:
H4N = open("Harvard IV_Negative Word List_Inf.txt", "r")

Inf = []
for line in H4N:
    stripped_line = line.strip()
    Inf.append(stripped_line)

H4N.close()

In [166]:
Inf = [x.lower() for x in Inf]
Inf = list(set(lemmatize_words(Inf)))

In [172]:
prop_list = []
H4N_list = []
for company, _ in filling_lem.items():
    for date, val in filling_lem[company].items():
        c = 0
        h = 0
        for i in val:
            if i in word_list:
                c+=1
            if i in Inf:
                h+=1
        l = len(val)
        if l==0:
            prop_list.append([company,date,0])
            H4N_list.append([company,date,0])
        else:
            prop_list.append([company,date,c/len(val)])
            H4N_list.append([company,date,h/len(val)])

In [173]:
filehandler = open("prop_df","wb")
pickle.dump(pd.DataFrame(prop_list),filehandler)
filehandler.close()

In [174]:
filehandler = open("prop_H4N_df","wb")
pickle.dump(pd.DataFrame(H4N_list),filehandler)
filehandler.close()

## TFIDF

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
N = 0
for company, _ in filling_lem.items():
    N+= len(_)  

In [92]:
N

11808

In [104]:
df_dict = {}
aj_dict = {}

In [106]:
for company, _ in filling_lem.items():
    aj_dict[company] = {}
    for date, val in filling_lem[company].items():
        aj_dict[company][date] = len(val)
        for i in set(val):
            if i not in df_dict:
                df_dict[i] = 1
            else:
                df_dict[i] += 1

In [111]:
wij_dict = {}

In [116]:
for company, _ in filling_lem.items():
    wij_dict[company] = {}
    for date, val in filling_lem[company].items():
        wij_dict[company][date] = {}
        for i in set(val):
            wij_dict[company][date][i] = (1+np.log(val.count("item")))/(1+np.log(aj_dict[company][date]))*np.log(N/df_dict[i])

In [122]:
filehandler = open("weight_dict.obj","wb")
pickle.dump(wij_dict,filehandler)
filehandler.close()

In [123]:
with open('weight_dict.obj', 'rb') as handle:
    wij_dict = pickle.load(handle)

In [124]:
len(wij_dict["NWL"]["2021-02-19"])

3248

In [128]:
tfidf_list = []
for company, _ in filling_lem.items():
    for date, val in filling_lem[company].items():
        c = 0
        d = 0
        for i in val:
            d += wij_dict[company][date][i] 
            if i in word_list:
                c+=wij_dict[company][date][i] 
        if len(val)==0:
            tfidf_list.append([company, date, 0])
        else:
            tfidf_list.append([company, date, c/d])

In [131]:
filehandler = open("tfidf_df","wb")
pickle.dump(pd.DataFrame(tfidf_list),filehandler)
filehandler.close()

In [132]:
pd.DataFrame(tfidf_list)

,0,1,2
0,NWL,2021-02-19,0.05166388
1,NWL,2020-03-02,0.05376505
2,NWL,2019-03-04,0.04501598
3,NWL,2018-03-01,0.03255259
4,NWL,2017-03-01,0.02908618
...,...,...,...
11803,SEDG,2018-05-10,0.01779219
11804,SEDG,2017-11-08,0.01912708
11805,SEDG,2017-08-03,0.01941586
11806,SEDG,2017-05-10,0.02103066


In [178]:
tfidf_H4N_list = []
for company, _ in filling_lem.items():
    for date, val in filling_lem[company].items():
        c = 0
        d = 0
        for i in val:
            d += wij_dict[company][date][i] 
            if i in Inf:
                c+=wij_dict[company][date][i] 
        if len(val)==0:
            tfidf_H4N_list.append([company, date, 0])
        else:
            tfidf_H4N_list.append([company, date, c/d])

In [179]:
filehandler = open("tfidf_H4N_df","wb")
pickle.dump(pd.DataFrame(tfidf_H4N_list),filehandler)
filehandler.close()

In [181]:
t = pd.DataFrame(tfidf_H4N_list)

In [184]:
sum(t[2]==0)

765